Importing Libs

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Loading Data

In [18]:
profiles_df = pd.read_csv('C:\\Users\\LENOVO\\Cloned repos\\ml-pipeline-formative\\ML-Pipeline-formative-2\\data\\customer_social_profiles - customer_social_profiles.csv')
transactions_df = pd.read_csv('C:\\Users\\LENOVO\\Cloned repos\\ml-pipeline-formative\\ML-Pipeline-formative-2\\data\\customer_transactions - customer_transactions.csv')
profiles_df.head(5)
transactions_df.head(5)

,customer_id,transaction_id,purchase_amount,purchase_date,product_category,customer_rating
0,151,1001,408,1/1/2024,Sports,2.3
1,192,1002,332,1/2/2024,Electronics,4.2
2,114,1003,442,1/3/2024,Electronics,2.1
3,171,1004,256,1/4/2024,Clothing,2.8
4,160,1005,64,1/5/2024,Clothing,1.3


Dataset Merge

In [19]:
profiles_df['customer_id'] = profiles_df['customer_id'].str.replace('A', '').astype(int)
merged_df = pd.merge(transactions_df, profiles_df, on='customer_id', how='inner')
merged_df.head(5)

,customer_id,transaction_id,purchase_amount,purchase_date,product_category,customer_rating,social_media_platform,engagement_score,purchase_interest_score,review_sentiment
0,151,1001,408,1/1/2024,Sports,2.3,TikTok,61,1.3,Neutral
1,151,1001,408,1/1/2024,Sports,2.3,Twitter,72,1.6,Neutral
2,151,1001,408,1/1/2024,Sports,2.3,Twitter,82,3.6,Negative
3,192,1002,332,1/2/2024,Electronics,4.2,Instagram,60,4.3,Positive
4,114,1003,442,1/3/2024,Electronics,2.1,Facebook,87,4.8,Negative


In [20]:
import pillow_heif

# ensure pillow_heif is registered so PIL can open HEIC files
pillow_heif.register_heif_opener()

def load_image(path=r'C:\Users\LENOVO\Cloned repos\ml-pipeline-formative\ML-Pipeline-formative-2\data\images\antony_neutral.heic', size=None):
    """
    Load an image (HEIC supported via pillow_heif).
    - path: path to the image (string)
    - size: optional (width, height) tuple to resize the image
    Returns a PIL.Image.Image instance.
    """
    image = Image.open(path)
    if size is not None:
        image = image.resize(size)
    return image

def augmen

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              219 non-null    int64  
 1   transaction_id           219 non-null    int64  
 2   purchase_amount          219 non-null    int64  
 3   purchase_date            219 non-null    object 
 4   product_category         219 non-null    object 
 5   customer_rating          200 non-null    float64
 6   social_media_platform    219 non-null    object 
 7   engagement_score         219 non-null    int64  
 8   purchase_interest_score  219 non-null    float64
 9   review_sentiment         219 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 17.2+ KB


Save Merged Dataset

In [6]:
merged_df.to_csv('C:\\Users\\LENOVO\\Cloned repos\\ml-pipeline-formative\\ML-Pipeline-formative-2\\data\\merged_customer_data.csv', index=False)
print(f"Merged dataset saved successfully. Shape: {merged_df.shape}")

Merged dataset saved successfully. Shape: (219, 10)
